In [ ]:
import requests
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from time import sleep
import os

load_dotenv()  # Загружает переменные из файла .env

# Конфигурация
input_files = [
    "./1.csv",
    "./2.csv",
    "./3.csv",
    "./4.csv",
    "./5.csv",
    "./6.csv",
]
output_dir = "./out"

# Создаем директорию для результатов, если ее нет
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Базовый prompt
base_prompt = '''
Нужно определить принадлежность текста к одной или нескольким тематикам из заданного списка (Спорт, Юмор, Реклама, Соцсети, Политика, Личная жизнь) 
в соответствии с представленными ниже примерами.
Пример 1:
Пиняев, во, "Какой пеняев хороший, а! И Воробьев! Второй матч, второй гол! Супер, а! Уф!" Спорт: 1.0
Пример 2:
Недавно мы с женой ходили на концерт группы «32nd to Mars» и «Джарда Лета». Билеты на этот концерт я подарил жене на день рождения. Впервые в жизни угадал с подарком. Личная жизнь: 1.0
Пример 3:
комик , который прожил и осознал эту жизнь – Стас Старовойтов  Ищи ответы на все вопросы сегодня в НОВОМ СЕЗОНЕ « стендап » в 23:00 на ТНТ"	Юмор: 1.0
Пример 4:
КАКАЯ КАРТА УКАЖЕТ НА МАСИКА?  РАСПРОДАЖА КУРСА «ОРАКУЛ ЛЕНОРМАН. БАЗОВЫЙ КУРС» В РАССРОЧКУ	Реклама: 1.0
Пример 5:
Приглашаю всей в свое ВК сообщество, будем делиться опытом и травить байки!	Соцсети: 1.0
Пример 6:
Сегодня запланирована встреча председателя госдумы и представителей регионов	Политика: 1.0
Если текст относится к разным категориям, то пиши через черточку: Соцсети: 0.5/Политика: 0.5.
Если текст не принадлежит ни к одной из указанных категорий, пиши: Нет категории
При этом суммарная степень уверенности не должна быть больше 1.0. 
Теперь определи принадлежность к категории у этого текста:
'''

# Настройки OpenAI
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1"),
)
model_name = os.getenv("OPENAI_MODEL", "gpt-3.5-turbo")

def process_text(text, attempt=1):
    """Функция для обработки текста с помощью выбранного API"""
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {'role': 'system', 'content': base_prompt},
                {'role': 'user', 'content': text}
            ],
            temperature=0.7,
            max_tokens=2000,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❗ Ошибка OpenAI API: {e}")
        return None

def process_file(input_file):
    """Обработка одного файла"""
    file_name = Path(input_file).name
    output_file = Path(output_dir) / f"processed_{file_name}"
    
    print(f"\n🔹 Начинаем обработку файла: {file_name}")
    
    df = pd.read_csv(input_file)
    
    # Проверка наличия колонки для ответов
    if 'response' not in df.columns:
        df['response'] = ""
    
    # Подсчет обработанных
    processed_since_last_save = 0
    total_processed = 0
    
    for i, row in df.iterrows():
        # Пропуск уже обработанных строк
        if isinstance(df.at[i, 'response'], str) and df.at[i, 'response'].strip():
            print(f"⏭️ Строка {i + 1} уже обработана, пропускаем.")
            continue

        # Конкатенация всех столбцов (кроме response) в одну строку
        text_parts = []
        for col in df.columns:
            if col != 'response' and pd.notna(row[col]):
                text_parts.append(str(row[col]))
        text = ' '.join(text_parts).strip()

        if not text:
            print(f"⚠️ Строка {i + 1} пустая, пропускаем.")
            continue

        print(f"🧠 Обработка строки {i + 1}...")
        
        for attempt in range(1, 4):  # 3 попытки на обработку
            response = process_text(text, attempt)
            if response is not None:
                df.at[i, 'response'] = response
                print(f"✅ [{attempt} попытка] Ответ записан в строку {i + 1}")
                print(f"Результат: {response[:100]}...")  # Выводим первые 100 символов
                processed_since_last_save += 1
                total_processed += 1
                break
            else:
                wait_time = attempt * 5
                print(f"🔄 [{attempt} попытка] Ошибка, пробуем снова через {wait_time} сек...")
                sleep(wait_time)
        else:
            print(f"❌ Не удалось обработать строку {i + 1} после 3 попыток")
            df.at[i, 'response'] = "Ошибка обработки"
            processed_since_last_save += 1
            total_processed += 1

        # Промежуточное сохранение каждые 10 строк
        if processed_since_last_save >= 10:
            df.to_csv(output_file, index=False)
            print(f"💾 Промежуточное сохранение. Обработано строк: {total_processed}")
            processed_since_last_save = 0
            sleep(1)  # Пауза между запросами

    # Финальное сохранение файла
    df.to_csv(output_file, index=False)
    print(f"\n✅ Файл {file_name} обработан. Результаты сохранены в: {output_file}")
    print(f"Всего обработано строк: {total_processed}")
    return output_file

# Обработка всех файлов
processed_files = []
for input_file in input_files:
    if Path(input_file).exists():
        try:
            result_file = process_file(input_file)
            processed_files.append(result_file)
        except Exception as e:
            print(f"🔥 Критическая ошибка при обработке файла {input_file}: {e}")
    else:
        print(f"⚠️ Файл {input_file} не найден, пропускаем")

print("\n🎉 Обработка завершена!")
print("Обработанные файлы:")
for file in processed_files:
    print(f"- {file}")


🔹 Начинаем обработку файла: 1.csv
🧠 Обработка строки 1...
✅ [1 попытка] Ответ записан в строку 1
Результат: Нет категории...
🧠 Обработка строки 2...
✅ [1 попытка] Ответ записан в строку 2
Результат: Нет категории...
🧠 Обработка строки 3...
✅ [1 попытка] Ответ записан в строку 3
Результат: Соцсети: 1.0...
🧠 Обработка строки 4...
✅ [1 попытка] Ответ записан в строку 4
Результат: Соцсети: 0.7/Политика: 0.3...
🧠 Обработка строки 5...
✅ [1 попытка] Ответ записан в строку 5
Результат: Соцсети: 0.6/Юмор: 0.4...
🧠 Обработка строки 6...
✅ [1 попытка] Ответ записан в строку 6
Результат: Соцсети: 0.7/Политика: 0.3...
🧠 Обработка строки 7...
✅ [1 попытка] Ответ записан в строку 7
Результат: Реклама: 0.7/Личная жизнь: 0.3...
🧠 Обработка строки 8...
✅ [1 попытка] Ответ записан в строку 8
Результат: Спорт: 1.0...
🧠 Обработка строки 9...
✅ [1 попытка] Ответ записан в строку 9
Результат: Спорт: 0.7/Соцсети: 0.3...
🧠 Обработка строки 10...
✅ [1 попытка] Ответ записан в строку 10
Результат: Спорт: 0.6/С

In [5]:
# Загрузка файла
df2 = pd.read_csv(file_path)
df2

,doc_text,image2text,speech2text,response
0,"Ребят, врубайте Матч ТВ🎙️ Тут мастер эфира Ру...",NaN,NaN,Спорт: 0.5/Юмор: 0.5
1,https://www.kinopoisk.ru/film/5203757/,"стальное сердце гика, приключения г посия лить...",NaN,Нет категории
2,ЛЕНОРМАН ПОМОГ ВЕРНУТЬ ДЕТЕЙ Мои Волшебные ✨ ...,~ 1 1 1 2 28 ` 7,"Мои волшебные, расскажу и покажу вам, как орак...",Реклама: 0.5/Соцсети: 0.5
3,Пока обратила внимание на суровый самарский не...,афтастьском 21 secuscoless,NaN,Нет категории
4,Фото @me.lenn Макияж @chaika_beauty Одежда @vl...,NaN,NaN,Соцсети: 1.0
...,...,...,...,...
995,Вспоминаем любимые шутки) Смотрите Женский ст...,и я такая - это не холодец! это слайм из свиньи!,NaN,Юмор: 0.7/Реклама: 0.3
996,tashcin https://t.me/tashcin/1713 Матч года...,сектор/ 98 2473 arena pint juice цска арена 20...,NaN,Спорт: 1.0
997,⚡️🗞️ Сразу две новых публикации в СМИ с Алексе...,новая публикация в сми ( причину побега захаро...,NaN,Соцсети: 0.5/Политика: 0.5\n\nОбоснование:\n- ...
998,tashcin https://t.me/tashcin/1713 Матч года...,сектор/ 98 2473 arena pint juice цска арена 20...,NaN,Спорт: 1.0
